In [1]:
import random
from scipy.misc import imrotate, imread, imsave, imresize

def flip(image, y):
    image = np.fliplr(np.array(image))
    return image, y

def rotate(image, y):
    angle = random.uniform(-13.0,13.0)
    image = imrotate(np.array(image), angle)
    return image, y

def crop(image, y):
    image = np.array(image)
    
    from_x = int(image.shape[1]*0.075)
    to_x = image.shape[1] - from_x
    
    from_y = int(image.shape[0]*0.075)
    to_y = image.shape[0] - from_y
    
    probability = random.randint(3,5)
    if probability==0:
        image = image[from_y:,from_x:,: ] 
    elif probability==1:
        image = image[:,from_x:,: ] 
    elif probability==2:
        image = image[from_y: ,: ] 
    elif probability==3:
        image = image[ :to_y, : ] 
    elif probability==4:
        image = image[ : , :to_x, ] 
    else :
        image = image[:to_y, :to_x, ]   
    return image, y

def data_augmentation_generator(path_features, labels_csv, batch_size, sample_size):
    img_rows, img_cols = 224, 224
    
    filenames = list( np.array([list(labels_csv.keys())[i] for i in range(2500) if (i+1)%25!=0 ]) )
    sample_size = 2400
    #filenames = list(labels_csv.keys())[:sample_size]
    
    while True:
        random.shuffle(filenames)
        for iterator in range(sample_size // batch_size):
            batch_features = np.zeros((batch_size, img_rows, img_cols, 3))
            batch_labels = np.zeros((batch_size, 50))
            
            for i in range(0, batch_size):
                # read picture
                filename = join(path_features, filenames[i + iterator * batch_size])
                pict = imread(filename, mode='RGB')
                
                #img = image.load_img(filename, target_size=(224, 224))
                #x = image.img_to_array(img)
                #x = np.expand_dims(x, axis=0)
                #pict = preprocess_input(x)
                
                coords = labels_csv[filenames[i + iterator * batch_size]]
                
                probability = random.randint(0,1)
                if probability == 1:
                    pict, coords = flip(pict, coords)
                
                probability = random.randint(0,1)
                if probability == 1:
                    pict, coords = rotate(pict, coords)
                
                probability = random.randint(0,1)
                if probability == 1:
                    pict, coords = crop(pict, coords)
                    
                pict = imresize(pict, (img_rows, img_cols), interp = 'bilinear')
                
                batch_features[i] = pict
                batch_labels[i][coords] = 1
            
            #batch_features = batch_features.astype('float32')# / 255.0
            batch_features = preprocess_input(batch_features)
            yield (batch_features, batch_labels)

In [2]:

from keras.applications import ResNet50
from keras.applications import InceptionV3
from keras.applications import Xception # TensorFlow ONLY
from keras.applications import VGG16
from keras.applications import VGG19

Using TensorFlow backend.


In [3]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

# img_path = 'elephant.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# preds = model.predict(x)
# # decode the results into a list of tuples (class, description, probability)
# # (one such list for each sample in the batch)
# print('Predicted:', decode_predictions(preds, top=3)[0])
# # Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]

In [4]:
from keras.layers import Dense
from keras.models import Model

model.layers.pop()
x =  Dense(50, activation='softmax')(model.layers[-1].output)
# x =  Dense(50, activation='softmax')(model.output)

final_model = Model(input = model.input, output = x)

/home/tamerlan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [5]:
final_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 230, 230, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
___________________________________________________________________________________________

In [6]:
from os.path import join
def read_csv(filename):
    res = {}
    with open(filename) as fhandle:
        next(fhandle)
        for line in fhandle:
            filename, class_id = line.rstrip('\n').split(',')
            res[filename] = int(class_id)
    return res

In [7]:
gt_dir = '/home/tamerlan/CV/task5/tests/00_gt'
train_dir = '/home/tamerlan/CV/task5/tests/00_input/train/images'
train_gt = read_csv(join(gt_dir, 'gt.csv'))

In [8]:
for layer in final_model.layers[:150]:
    layer.trainable = False

In [9]:
for layer in final_model.layers[150:]:
    layer.trainable = True

In [12]:
from keras.optimizers import SGD
sgd = SGD(lr=0.005)
final_model.compile(optimizer = sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
final_model.fit_generator(data_augmentation_generator(train_dir, train_gt, 16, 2500), 
                          steps_per_epoch=2400//8,  
                          epochs=2,
                          verbose=1,
                         validation_data = (data,labels ))
final_model.save('birds_model_valid.hdf5')

Epoch 1/2
300/300 [==============================] - 7554s - loss: 0.1422 - acc: 0.9819 - val_loss: 0.7821 - val_acc: 0.7900
Epoch 2/2
300/300 [==============================] - 7524s - loss: 0.1009 - acc: 0.9877 - val_loss: 0.7533 - val_acc: 0.8200


In [18]:
final_model.fit_generator(data_augmentation_generator(train_dir, train_gt, 16, 2500), 
                          steps_per_epoch=2400//16,  
                          epochs=2,
                          verbose=1,
                         validation_data = (data,labels ))
final_model.save('birds_model_valid2.hdf5')

Epoch 1/2
150/150 [==============================] - 3741s - loss: 0.0577 - acc: 0.9954 - val_loss: 0.7568 - val_acc: 0.8100
Epoch 2/2
150/150 [==============================] - 3774s - loss: 0.0507 - acc: 0.9962 - val_loss: 0.7191 - val_acc: 0.8200


In [26]:
final_model.fit_generator(data_augmentation_generator(train_dir, train_gt, 32, 2500), 
                          steps_per_epoch=2400//32,  
                          epochs=2,
                          verbose=1,
                         validation_data = (data,labels ))
final_model.save('birds_model_valid_new.hdf5')

Epoch 1/2
75/75 [==============================] - 1990s - loss: 0.2224 - acc: 0.9646 - val_loss: 0.7331 - val_acc: 0.8000
Epoch 2/2
75/75 [==============================] - 2018s - loss: 0.1842 - acc: 0.9771 - val_loss: 0.7250 - val_acc: 0.7800


In [20]:
final_model.fit_generator(data_augmentation_generator(train_dir, train_gt, 32, 2500), 
                          steps_per_epoch=2400//32,  
                          epochs=4,
                          verbose=1,
                         validation_data = (data,labels ))
final_model.save('birds_model_valid_new.hdf5')

Epoch 1/4
75/75 [==============================] - 1387s - loss: 0.7008 - acc: 0.8483 - val_loss: 0.9405 - val_acc: 0.7300
Epoch 2/4
75/75 [==============================] - 1380s - loss: 0.6122 - acc: 0.8675 - val_loss: 0.9204 - val_acc: 0.7200
Epoch 3/4
75/75 [==============================] - 1380s - loss: 0.5301 - acc: 0.8871 - val_loss: 0.8622 - val_acc: 0.7700
Epoch 4/4
75/75 [==============================] - 1392s - loss: 0.4754 - acc: 0.8942 - val_loss: 0.8633 - val_acc: 0.7100


In [10]:
final_model.fit_generator(data_augmentation_generator(train_dir, train_gt, 32, 2500), 
                          steps_per_epoch=2400//32,  
                          epochs=1,
                          verbose=1)
                         #validation_data = (data[test],labels[test]) )
final_model.save('birds_model_valid.hdf5')

Epoch 1/1
75/75 [==============================] - 4522s - loss: 0.1563 - acc: 0.9792   


In [11]:
final_model.fit_generator(data_augmentation_generator(train_dir, train_gt, 32, 2500), 
                          steps_per_epoch=2400//32,  
                          epochs=1,
                          verbose=1)
                         #validation_data = (data[test],labels[test]) )
final_model.save('birds_model_valid.hdf5')

Epoch 1/1
75/75 [==============================] - 4830s - loss: 0.1259 - acc: 0.9879   


In [ ]:
final_model.fit_generator(data_augmentation_generator(train_dir, train_gt, 32, 2500), 
                          steps_per_epoch=2400//32,  
                          epochs=1,
                          verbose=1,
                         validation_data = (data,labels ))
final_model.save('birds_model_valid.hdf5')

Epoch 1/1
 1/75 [..............................] - ETA: 8790s - loss: 0.0910 - acc: 1.0000

In [14]:
final_model.fit_generator(data_augmentation_generator(train_dir, train_gt, 32, 2500), 
                          steps_per_epoch=2400//32,  
                          epochs=1,
                          verbose=1)
                         #validation_data = (data[test],labels[test]) )
final_model.save('birds_model_valid.hdf5')

Epoch 1/1
75/75 [==============================] - 3704s - loss: 0.2733 - acc: 0.9479   


In [15]:
final_model.fit_generator(data_augmentation_generator(train_dir, train_gt, 32, 2500), 
                          steps_per_epoch=2400//32,  
                          epochs=1,
                          verbose=1)
                         #validation_data = (data[test],labels[test]) )
final_model.save('birds_model_valid.hdf5')

Epoch 1/1
75/75 [==============================] - 3674s - loss: 0.2340 - acc: 0.9596   


In [16]:
final_model.fit_generator(data_augmentation_generator(train_dir, train_gt, 32, 2500), 
                          steps_per_epoch=2400//32,  
                          epochs=1,
                          verbose=1)
                         #validation_data = (data[test],labels[test]) )
final_model.save('birds_model_valid.hdf5')

Epoch 1/1
75/75 [==============================] - 3655s - loss: 0.1992 - acc: 0.9679   


In [17]:
final_model.fit_generator(data_augmentation_generator(train_dir, train_gt, 32, 2500), 
                          steps_per_epoch=2400//32,  
                          epochs=1,
                          verbose=1)
                         #validation_data = (data[test],labels[test]) )
final_model.save('birds_model_valid.hdf5')

Epoch 1/1
75/75 [==============================] - 3671s - loss: 0.1772 - acc: 0.9712   


In [11]:
#final_model.save('birds_model_valid.hdf5')

In [22]:
from keras.models import load_model

final_model = load_model('birds_model_valid.hdf5')

In [22]:
from keras.models import load_model

fml = load_model('model.hdf5')

ValueError: Optimizer weight shape (256,) not compatible with provided weight shape (1, 1, 64, 64)

In [11]:
filenames = list(train_gt.keys())
data = np.zeros((len(filenames), 224, 224,3))
for i in range(0, len(train_gt.keys())):
        filename = join(train_dir, filenames[i])
        pict = imread(filename)
        if len(pict.shape)==2:
            temp_pict = np.zeros((pict.shape[0],pict.shape[1],3))
            temp_pict[...,0] = pict
            temp_pict[...,1] = pict
            temp_pict[...,2] = pict
            pict = temp_pict
        pict = imresize(pict , (224, 224), interp = 'bicubic')
        data[i, :] = pict
data = preprocess_input(data)

In [12]:
labels = np.zeros((len(filenames), 50))
j = 0
for i in train_gt.values():
    labels[j][i] = 1
    j = j + 1
    
ind = np.arange((len(filenames)))
np.random.shuffle(ind)

In [13]:
test = np.array([i for i in range(2500) if (i+1)%25==0 ]) 

In [14]:
data = data[test]
labels = labels[test]

In [23]:
scores = final_model.evaluate(data,labels, verbose=1)
print(scores)
print("Точность : %.5f" % (scores))

100/100 [==============================] - 61s    
[0.75330213904380794, 0.81999999999999995]


TypeError: must be real number, not list

In [ ]:
y_pred = final_model.predict(data[ind[:100]])

In [ ]:
np.where(abs(labels[ind[:100]] - y_pred) == 0 , 0, 1).sum(axis=1)

In [ ]:
j = 0
for i in np.argmax(y_pred, axis = 1):
    y_pred[j] = np.zeros((1,50))
    y_pred[j][i] = 1
    j = j + 1
    

In [ ]:
y_pred

In [ ]:
img_path = '/home/tamerlan/CV/task5/tests/00_input/train/images/0000.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)

plt.imshow(x)
plt.show()

x = np.expand_dims(x, axis=0)
x = preprocess_input(x)


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(x[0])
plt.show()

In [ ]:
x.shape

In [ ]:
filenames = list(train_gt.keys())[:100]

In [ ]:
np.array([i for i in range(2500) if (i+1)%25!=0 ]) 

In [ ]:
filenames = []

In [ ]:
tyfilenames

In [15]:
test

array([  24,   49,   74,   99,  124,  149,  174,  199,  224,  249,  274,
        299,  324,  349,  374,  399,  424,  449,  474,  499,  524,  549,
        574,  599,  624,  649,  674,  699,  724,  749,  774,  799,  824,
        849,  874,  899,  924,  949,  974,  999, 1024, 1049, 1074, 1099,
       1124, 1149, 1174, 1199, 1224, 1249, 1274, 1299, 1324, 1349, 1374,
       1399, 1424, 1449, 1474, 1499, 1524, 1549, 1574, 1599, 1624, 1649,
       1674, 1699, 1724, 1749, 1774, 1799, 1824, 1849, 1874, 1899, 1924,
       1949, 1974, 1999, 2024, 2049, 2074, 2099, 2124, 2149, 2174, 2199,
       2224, 2249, 2274, 2299, 2324, 2349, 2374, 2399, 2424, 2449, 2474,
       2499])

In [13]:
len(final_model.layers)

177